# SaRNet: A Dataset for Deep Learning Assisted Search and Rescue with Satellite Imagery


**Abstract**

 Access to high resolution satellite imagery has dramatically increased in recent years as several new constellations have entered service. High revisit frequencies as well as improved resolution has widened the use cases of satellite imagery to areas such as humanitarian relief and even Search and Rescue (SaR). We propose a novel remote sensing object detection dataset for deep learning assisted SaR. This dataset contains only very small objects that have been identified as potential targets in a live search and rescue scenario. We evaluate the application of popular object detection frameworks to this dataset as a baseline to inform further research. We also propose a novel object detection metric, specifically designed to be used in a deep learning assisted SaR setting. 

## Download the data

In [ ]:
!wget https://michaeltpublic.s3.amazonaws.com/sarnet.zip

In [ ]:
!unzip -q -o sarnet.zip 

## Install Dependencies

In [ ]:
# installing detectron2 with torch version 1.9
import torch
assert torch.__version__.startswith("1.9")   # check the torch version is the default, this may require changing in the future
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html

# after install you need to restart the runtime

# Setup Data Loaders

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.engine import DefaultTrainer

from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

from detectron2.data.datasets import register_coco_instances
try:
  register_coco_instances("dataset_train_1", {}, "/content/dataset/annotations/train.json", "/content/dataset/train")
  register_coco_instances("dataset_val_1", {}, "/content/dataset/annotations/val.json", "/content/dataset/val")
  register_coco_instances("dataset_test_1", {}, "/content/dataset/annotations/test.json", "/content/dataset/test")
except AssertionError:
  print("INFO: Datasets already registered.")

para_metadata = MetadataCatalog.get("dataset_train_1")
dataset_dicts = DatasetCatalog.get("dataset_train_1")

para_metadata_val = MetadataCatalog.get("dataset_val_1")
dataset_dicts_val = DatasetCatalog.get("dataset_val_1")

para_metadata_test = MetadataCatalog.get("dataset_test_1")
dataset_dicts_test = DatasetCatalog.get("dataset_test_1")

# Training

In [ ]:
from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader
import detectron2.utils.comm as comm
import torch
import time
import datetime
import logging
import numpy as np

class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):
            if np.random.rand() > 0.1:
              continue
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            loss_batch = self._get_loss(inputs)
            
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        print("validation loss on 10% of val data: {}".format(mean_loss))
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses
            
    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
        
    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)
                     
    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            cfg.TEST.EVAL_PERIOD_LOSS,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks

**faster_rcnn_R_50_DC5_3x.yaml**

In [ ]:
# do configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_DC5_3x.yaml"))
cfg.OUTPUT_DIR = "faster_rcnn_R_50_DC5_3x"
cfg.DATASETS.TRAIN = ("dataset_train_1",)
cfg.DATASETS.TEST = ("dataset_val_1",)
cfg.TEST.EVAL_PERIOD = 1000  # do a full coco evaluation every 500 iterations
cfg.TEST.EVAL_PERIOD_LOSS = 500 # calculate the validation loss every 100 iterations
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_DC5_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0001
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.RPN.IOU_THRESHOLDS = [0.2, 0.4]



os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

**faster_rcnn_R_50_C4_3x.yaml**


In [ ]:
# do configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_C4_3x.yaml"))
cfg.OUTPUT_DIR = "faster_rcnn_R_50_C4_3x.yaml"
cfg.DATASETS.TRAIN = ("dataset_train_1",)
cfg.DATASETS.TEST = ("dataset_val_1",)
cfg.TEST.EVAL_PERIOD = 1000  # do a full coco evaluation every 500 iterations
cfg.TEST.EVAL_PERIOD_LOSS = 500 # calculate the validation loss every 100 iterations
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_C4_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0001
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.RPN.IOU_THRESHOLDS = [0.2, 0.4]



os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

**faster_rcnn_R_50_FPN_3x.yaml**

In [ ]:
# do configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.OUTPUT_DIR = "faster_rcnn_R_50_FPN_3x.yaml"
cfg.DATASETS.TRAIN = ("dataset_train_1",)
cfg.DATASETS.TEST = ("dataset_val_1",)
cfg.TEST.EVAL_PERIOD = 1000  # do a full coco evaluation every 500 iterations
cfg.TEST.EVAL_PERIOD_LOSS = 500 # calculate the validation loss every 100 iterations
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0001
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.RPN.IOU_THRESHOLDS = [0.2, 0.4]



os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

## Evaluation
The performance criteria for this application is less strict than for other applications, more false positives are acceptable. We will look at the number of predictions vs the recall.

In [ ]:
%matplotlib inline
para_metadata = MetadataCatalog.get("dataset_test_1")
dataset_dicts = DatasetCatalog.get("dataset_test_1")
import matplotlib.pyplot as plt
for d in random.sample(dataset_dicts, 10):
  img = cv2.imread(d["file_name"])
  
  # Inference should use the config with parameters that are used in training
  # cfg now already contains everything we've set previously. We changed it a little bit for inference:
  cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3   # set a custom testing threshold
  cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.1
  predictor = DefaultPredictor(cfg)
  outputs = predictor(img)

  # convert image for display only
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  v = Visualizer(img[:, :, ::-1], para_metadata)
  v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  fig = plt.figure(figsize=(15,15))
  plt.imshow(v.get_image()[:, :, ::-1])


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from detectron2.structures import Boxes, BoxMode, pairwise_iou

Nsamples = 50

outputDirs = ["faster_rcnn_R_50_FPN_3x.yaml", "faster_rcnn_R_50_C4_3x.yaml", "faster_rcnn_R_50_DC5_3x.yaml"]
modelNames = ["faster_rcnn_R_50_FPN_3x", "faster_rcnn_R_50_C4_3x", "faster_rcnn_R_50_DC5_3x"]


gt_boxes = [[],[],[]]
pred_boxes = [[],[],[]]
pred_scores = [[],[],[]]
for k, outputDir in enumerate(outputDirs):
  for j,d in enumerate(random.sample(dataset_dicts, Nsamples)):
    img = cv2.imread(d["file_name"])

    yamlFile = outputDir
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/" + yamlFile))
    cfg.OUTPUT_DIR = yamlFile
   
    # cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
    
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.MODEL.RPN.IOU_THRESHOLDS = [0.2, 0.4]

    # Inference should use the config with parameters that are used in training
    # cfg now already contains everything we've set previously. We changed it a little bit for inference:
    cfg.MODEL.WEIGHTS = os.path.join(outputDir, "model_final.pth")  # path to the model we just trained
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.01   # set a custom testing threshold
    cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.1
    predictor = DefaultPredictor(cfg)
    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")


    gtBoxArray = np.zeros((len(d['annotations']), 4))

    for i,annotation in enumerate(d['annotations']):
      gtBoxArray[i] = np.array(annotation['bbox'])

    # reformat ground truth boxes
    gtBoxArray = BoxMode.convert(gtBoxArray, BoxMode.XYWH_ABS, BoxMode.XYXY_ABS)
    
    # prediction boxes - dont reformat
    # predBoxArray = BoxMode.convert(instances.pred_boxes.tensor.numpy(), BoxMode.XYXY_ABS, BoxMode.XYWH_ABS)
    predBoxArray = instances.pred_boxes.tensor.numpy()

    gt_boxes[k].append(gtBoxArray)
    pred_boxes[k].append(predBoxArray)
    pred_scores[k].append(instances.scores.numpy())

    print("testing: {}/{}".format(j,Nsamples))






In [ ]:
#IoU threshold
iou_threshold = 0.1

ground_truth_boxes = np.zeros((len(outputDirs)))
thresholds = np.linspace(0,1,100)
correct_boxes = np.zeros((len(outputDirs),thresholds.shape[0]))
prediction_boxes = np.zeros((len(outputDirs),thresholds.shape[0]))

for k, outputDir in enumerate(outputDirs):
  for j,t in enumerate(thresholds):
    ground_truth_boxes[k] = 0
    for i, (gtBoxArray, predBoxArray, scores) in enumerate(zip(gt_boxes[k], pred_boxes[k], pred_scores[k])):
      # print("{}------------------------------:".format(i))
      gtBoxObjs = Boxes(gtBoxArray)
      predBoxObjs = Boxes(predBoxArray)
      overlaps = np.array(pairwise_iou(predBoxObjs, gtBoxObjs))
      # print(overlaps, scores)
      ground_truth_boxes[k] += gtBoxArray.shape[0]
      for score in scores:
        if score >= t:
          prediction_boxes[k][j] += 1

      # check for a correct prediction for each gt box
      for gt in range(overlaps.shape[1]):
        for predOverlap, score in zip(overlaps[:,gt], scores):
          # print(predOverlap, score)
          if predOverlap > iou_threshold and score > t:
            correct_boxes[k][j] += + 1

print(ground_truth_boxes, prediction_boxes, correct_boxes)



In [ ]:
modelNames = ["faster_rcnn_R_50_FPN_3x", "faster_rcnn_R_50_C4_3x", "faster_rcnn_R_50_DC5_3x"]


fig = plt.figure()
for k, outputDir in enumerate(modelNames):
  plt.plot(thresholds, prediction_boxes[k]/(Nsamples*0.25))  

plt.legend(modelNames)
plt.xlim([0.05, 1.0])
plt.ylim([-5, 100])
plt.title("Detections Density - Faster R-CNN")
plt.xlabel("Threshold")
plt.ylabel("Detections per sq km")

fig = plt.figure()
for k, outputDir in enumerate(modelNames):
  plt.plot(thresholds, 100*correct_boxes[k]/ground_truth_boxes[k])

plt.legend(modelNames)
plt.xlim([0.05, 1.0])
plt.title("Recall on conservative labels - Faster R-CNN")
plt.ylabel("Recall %")
plt.xlabel("Threshold")


styles = ['-', '--', '-.']

fig, ax1 = plt.subplots()
plt.title("Detection Density and Recall - Faster R-CNN Baseline")
color = 'tab:blue'
ax1.set_xlabel('Confidence Threshold')
ax1.set_ylabel('Detections per sq km', color=color)
for k, outputDir in enumerate(modelNames):
  ax1.plot(thresholds, prediction_boxes[k]/(Nsamples*0.25), color=color, linestyle=styles[k])
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_xlim([0.01, 1.0])
ax1.set_ylim([-5, 100])
ax1.legend(modelNames)
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('Recall %', color=color)  # we already handled the x-label with ax1

for k, outputDir in enumerate(modelNames):
  ax2.plot(thresholds, 100*correct_boxes[k]/ground_truth_boxes[k], color=color, linestyle=styles[k])

ax2.tick_params(axis='y', labelcolor=color)
ax2.set_xlim([0.01, 1.0])
ax2.set_ylim([-5, 100])
ax2.legend(modelNames)
plt.savefig("density_and_recall.png", dpi=300)
plt.savefig("density_and_recall.eps")
plt.show()

In [ ]:
ARd20 = np.zeros((len(modelNames)))

for l, outputDir in enumerate(modelNames):
  gts = ground_truth_boxes[l]
  correct = correct_boxes[l]
  densities = np.array(prediction_boxes[l])/(Nsamples*0.25)
  

  # find the lowest threshold that corresponds to a density lower than k
  for k in np.linspace(0,20, 21):
    r = correct[densities < k]
    if len(r) > 1:
      # ARd20[l] += t[0]/20
      r = 100*r[0]/gts
      ARd20[l] += r/20



for model, score in zip(modelNames, ARd20):
  print("Model:      {:30s}   ARd-20: {:.2f}".format(model, score))

    
